In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.4.1


In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


In [4]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [5]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences




In [12]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

#training
sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)
print(padded[0])

#testing
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)
print(testing_padded[0])

[   0    0    0   12   14   33  425  392   18   90   28    1    9   32
 1366 3585   40  486    1  197   24   85  154   19   12  213  329   28
   66  247  215    9  477   58   66   85  114   98   22 5675   12 1322
  643  767   12   18    7   33  400 8170  176 2455  416    2   89 1231
  137   69  146   52    2    1 7577   69  229   66 2933   16    1 2904
    1    1 1479 4940    3   39 3900  117 1584   17 3585   14  162   19
    4 1231  917 7917    9    4   18   13   14 4139    5   99  145 1214
   11  242  683   13   48   24  100   38   12 7181 5515   38 1366    1
   50  401   11   98 1197  867  141   10]
[  11  772 1498   12  252  235   11  217    2  366 6454    3   58   93
   11   90  102   11 1498  177   12  252   36    6 1126    1  674    7
 4387    1    4    1  327    7   36 8300  366    5 1403    1   13   29
   60   26    6  867  178   17    4 1037    5   12  227    3   79    4
  345   32  345 5159    5   10    6 1314 1143    2 5619    1    3    1
    5   10  173  322    7 1293 3938

In [7]:
type(word_index)
first_ten = dict(list(word_index.items())[:10])
first_ten

{'<OOV>': 1,
 'a': 4,
 'and': 3,
 'br': 8,
 'in': 9,
 'is': 7,
 'it': 10,
 'of': 5,
 'the': 2,
 'to': 6}

In [9]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
first_ten_reversed = dict(list(reverse_word_index.items())[:10])
first_ten_reversed



{1: '<OOV>',
 2: 'the',
 3: 'and',
 4: 'a',
 5: 'of',
 6: 'to',
 7: 'is',
 8: 'br',
 9: 'in',
 10: 'it'}

In [10]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [13]:
num_epochs = 10
model.fit(training_padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 8s 7ms/step - loss: 0.5989 - accuracy: 0.6410 - val_loss: 0.3510 - val_accuracy: 0.8465
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2409 - accuracy: 0.9091 - val_loss: 0.3858 - val_accuracy: 0.8341
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0898 - accuracy: 0.9794 - val_loss: 0.4592 - val_accuracy: 0.8293
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0212 - accuracy: 0.9979 - val_loss: 0.5428 - val_accuracy: 0.8272
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0056 - accuracy: 0.9996 - val_loss: 0.5944 - val_accuracy: 0.8276
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.6427 - val_accuracy: 0.8283
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 8.7047e-04 - accuracy: 1.0000 - val_loss: 0.6856 - val_accuracy: 0.8294
Ep

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [16]:
num_epochs = 10
model.fit(training_padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 6s 7ms/step - loss: 0.6449 - accuracy: 0.6960 - val_loss: 0.4049 - val_accuracy: 0.8348
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3632 - accuracy: 0.8536 - val_loss: 0.3342 - val_accuracy: 0.8560
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2812 - accuracy: 0.8853 - val_loss: 0.3272 - val_accuracy: 0.8592
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2363 - accuracy: 0.9085 - val_loss: 0.3368 - val_accuracy: 0.8557
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2090 - accuracy: 0.9226 - val_loss: 0.3525 - val_accuracy: 0.8514
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1905 - accuracy: 0.9304 - val_loss: 0.3755 - val_accuracy: 0.8461
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1695 - accuracy: 0.9404 - val_loss: 0.3995 - val_accuracy: 0.8416
Epoch 

In [14]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [17]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [18]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
